# Fixed point square roots

## Newton's method

Perhaps the most obvious way of computing the square root of $\alpha$ is to iterate Newton's method to find a root of $f(x) = x^2 - \alpha$.
Specifically:
$$ x' = x - \frac{f(x)}{f'(x)} = x - \frac{x^2 - \alpha}{2 x} = \frac12 \left(x + \frac{\alpha}x\right)$$
We compute this in fixed point, approximating $x \approx N^{-1} y$, for some large integer constant $N$, with $y$ being the mantissa.
Equivalently, we approximate $\alpha \approx N^{-1} \beta$.

The update rule expressing $y'$ in terms of $y$ is therefore:
\begin{align}
y' &= N x' \\
&=\frac{N}2 \left( x + \frac{\alpha}x \right) \\
&=\frac{N}2 \left( N^{-1} y + \frac{N^{-1} \beta}{N^{-1} y} \right) \\
&=\frac12 \left( y + \frac{N \beta}{y} \right) \\
\end{align}

In particular, with $N = 2^n$, we get the following:

    y' = (y + (beta << n) / y) >> 1

If `y` and `beta` are stored with some given precision, then the division must be evaluated at twice said precision.
Thus, for 32-bit values we require a 64-bit division.

## Initial estimate

Some initial value $x_0$ must be selected, hopefully as close as possible to $\sqrt{\alpha}$.
Here we choose the initial value for $y$ by using some bit-masking on $\beta$'s value.
The key observation is that the value of `y` which is the correct fixed-point square root of `beta` will have about half as many leading zeroes, for small values of `beta`.
Thus, by taking various chunks of the bits of `beta` and ORing them together we can yield an estimate with a leading 1 in approximately the right place.

To avoid numerical overflows in the evaluation of `y + (beta << n) / y` it is critical that `(beta << n) / y` not be too large, which requires that the error in `y` never be too high.
This occurs specifically when `beta` is large and we use too small of an initial value for `y`.
To prevent this, we guarantee that our bit-masking techniques include every bit of `beta` somewhere in our initial estimate of `y` shifted some non-negative amount to the left, so every value of `beta` yields a strictly larger initial `y`.

In [1]:
# Set things up with 32 bits, 29 of which are after the binary point.
# Thus, we represent dyadic rationals from 0 to precisely 7.99999999813735485076904296875.
# Our quantum increment (one ULP) is the precise number 0.00000000186264514923095703125.
bits = 32
N = 2**29
to_fixed = lambda z: int(z * N)
to_float = lambda m: m / float(N)

In [2]:
alpha = 6.7543
alpha = to_float(to_fixed(alpha)) # Round to the nearest representable fixed point value.
beta = to_fixed(alpha)
assert 0 <= beta < 2**bits, "Not enough bits to represent the argument!"

# Yield an initial estimate that is within a few orders of magnitude.
y = beta | (1 << 15)
y |= (beta & 0x000000ff) << 11
y |= (beta & 0x0000ffff) << 7
y |= (beta & 0x00ffffff) << 3
y |= (beta & 0x0fffffff) << 1

print "Starting value in quanta, y =", y, "   beta =", beta
for _ in xrange(7):
    y += (N * beta) / y
    assert 0 <= y < 2**bits, "Overflow in intermediate!"
    y >>= 1
    print "Fixed point value: %11i    decimal: %.8f    error in quanta: %i" % (y, to_float(y), y - to_fixed(alpha**0.5))

Starting value in quanta, y = 3649044416    beta = 3626187200
Fixed point value:  2091276213    decimal: 3.89530549    error in quanta: 696000460
Fixed point value:  1511094179    decimal: 2.81463224    error in quanta: 115818426
Fixed point value:  1399714228    decimal: 2.60717092    error in quanta: 4438475
Fixed point value:  1395282790    decimal: 2.59891672    error in quanta: 7037
Fixed point value:  1395275753    decimal: 2.59890361    error in quanta: 0
Fixed point value:  1395275753    decimal: 2.59890361    error in quanta: 0
Fixed point value:  1395275753    decimal: 2.59890361    error in quanta: 0


## Analysis
In the directory where this notebook appears there is also C code that exhaustively tests the above logic for every single possible `uint32_t` value.

For inputs with 32-bit fixed point, with 29 bits after the binary point (that is, representing values from 0 up to 7.99999999813735485076904296875 with a step size of precisely 0.00000000186264514923095703125), we get a result that is accurate to within two ULPs after 7 iterations of Newton's Method for every single input in the interval $[0, 7]$, where the result is exactly equal to the floor of the true answer down to the last bit (that is, the closest possible fixed-point approximation no greater than the true answer) in all but 384 of these cases.
For inputs in $(7, 8)$, we cannot guarantee non-overflows, but for all $2^{32} - 1$ possible non-zero `uint32_t` arguments we can guarantee that the above iteration never causes a division by zero. Of course, for `y = 0` we immediately divide by zero, so we special-case that value, yielding the following C code:

```C
// Evaluates the square root of beta, interpreted as fixed-point with a pre-multiplier of 2^29.
uint32_t square_root(uint32_t beta) {
    // Special case zero to avoid division by zero.
    if (beta == 0)
        return 0;
    // Produce an initial estimate.
    uint32_t y = beta | (1 << 15);
    y |= (beta & 0x000000ffu) << 11;
    y |= (beta & 0x0000ffffu) << 7;
    y |= (beta & 0x00ffffffu) << 3;
    y |= (beta & 0x0fffffffu) << 1;
    // Apply 7 rounds of Newton's method.
    uint64_t numerator = ((uint64_t)beta) << 29;
    for (int i = 0; i < 7; i++) {
        y += (uint32_t)(numerator / (uint64_t)y);
        y >>= 1;
    }
    return y;
}
```

We can get away with a mere 5 rounds of Newton's method by optimizing the shifted copies of `beta` we use to piece together our initial estimate of `y`.
More details appear at the end of this document.

## Avoiding long divisons

Assuming we don't have access to fast division of twice the integer precision we're storing our mantissas at, it is desirable to have an implementation that doesn't rely on having this additional precision.

TODO: Fill in an example of [this algorithm](https://en.wikipedia.org/wiki/Methods_of_computing_square_roots#Digit-by-digit_calculation).

# Annex: Initial estimate
If we instead use 13 shifted copies of `beta` to form `y`, we can get a sufficiently accurate initial estimate to only need 5 iterations of Newton's method to still maintain accuracy to within an ULP for every input in $[0, 7]$ (that is, all fixed point numbers between `0x00000000` and `0xe0000000`).
The code below spits out the code to produce the initial estimate of `y` from `beta`, for any number of bits and value of $N$.

In [3]:
bits = 32
N = 2**29
to_fixed = lambda z: int(z * N)
to_float = lambda m: m / float(N)

def shift_amount(b):
    import math
    fixed = (1 << b) - 1
    root = to_fixed(to_float(fixed)**0.5)
    return math.log(root, 2) - b

print "uint32_t y = beta;"
amounts = {}
for i in xrange(1, bits):
    shift = round(shift_amount(i))
    if shift > 0:
        # WARNING: You may need to change the formatting on these constants, depending on
        # your compiler. For example, if you bump `bits` up to 64, you may have to change
        # the constants to end with "llu" rather than just "u" to avoid truncation.
        amounts[shift] = "y |= (beta & 0x%08xu) << %i;" % ((1 << i) - 1, shift)
for key, value in sorted(amounts.items(), reverse=True):
    print value

uint32_t y = beta;
y |= (beta & 0x00000007u) << 13;
y |= (beta & 0x0000001fu) << 12;
y |= (beta & 0x0000007fu) << 11;
y |= (beta & 0x000001ffu) << 10;
y |= (beta & 0x000007ffu) << 9;
y |= (beta & 0x00001fffu) << 8;
y |= (beta & 0x00007fffu) << 7;
y |= (beta & 0x0001ffffu) << 6;
y |= (beta & 0x0007ffffu) << 5;
y |= (beta & 0x001fffffu) << 4;
y |= (beta & 0x007fffffu) << 3;
y |= (beta & 0x01ffffffu) << 2;
y |= (beta & 0x07ffffffu) << 1;
